# Probabilistic Classifier with Naive Bayes

## By Wilson Peguero Rosario

A Naive Bayes classifier is not a single algorithm but uses multiple machine learning algorithms to classify data. It not only uses probability, but it is simple to implement. Some real-world examples of its use include filtering spam, classifying documents, text analysis, or medical diagnosis.

To perform sentiment analysis using a Naive Bayes algorithm, complete the following:

1. Access the resources related to sentiment analysis, located in the topic Resources. Note: There are about 50 datasets that are suitable for use in a sentiment analysis task. For this part of the exercise, you must choose one of these datasets, provided it includes at least 10,000 instances.
2. Ensure that the datasets are suitable for classification using this method.
3. You may search for data in other repositories, such as Data.gov, Kaggle or Scikit Learn.

For your selected dataset, build a classification model as follows:

1. Explain the dataset and the type of information you wish to gain by applying a classification method.
2. Explain the Naive Bayes algorithm and how you will be using it in your analysis (list the steps, the intuition behind the mathematical representation, and address its assumptions).
3. Import the necessary libraries, then read the dataset into a data frame and perform initial statistical exploration.
4. Clean the data and address unusual phenomena (e.g., normalization, feature scaling, outliers); use illustrative diagrams and plots and explain them.
5. Formulate two questions that can be answered by applying a classification method using the Naïve Bayes.
6. Choose one of the Naive Bayes types of algorithms: Gaussian Naïve Bayes, Multinomial Naïve Bayes, or Bernoulli Naïve Bayes and explain your reasoning.
7. Split the data into dependent and independent variables (or features and labels).
8. Vectorize the text into numbers.
9. Train the Naive Bayes classifier on the training set.
10. Make classification predictions.
11. Interpret the results in the context of the questions you asked.
12. Validate your model using a confusion matrix, accuracy score, ROC-AUC curves, and k-fold cross validation. Then, explain the results.
13. Include all mathematical formulas used and graphs representing the final outcomes.

Prepare a comprehensive technical report as a markdown document or Jupyter notebook, including all code, code comments, all outputs, plots, and analysis. Make sure the project documentation contains a) problem statement, b) algorithm of the solution, c) analysis of the findings, and d) deferences.

### Solution

- Talk about the steam platform and it's ability to provide video game players the ability to play their favorite games not only on PCs, but also on steam authorized consoles (mainly sold by NVIDIA and VALVE).

The data set utilized is based on Steam Reviews that are prelabeled as either recommended or not recommended. The features in use will be the year in which the comment was posted as well as the user suggestion to classify unlabeled reviews on certain titles. This algorithm will also use other factors related to the game itself (such as the publisher) to increase the accuracy of the SVM algorithm.

In [1]:
import pandas as pd

df__texts = pd.read_csv('steam_reviews/train.csv')
df__game_info = pd.read_csv('steam_reviews/game_overview.csv')

df__steam_info = pd.merge(df__texts, df__game_info, on="title")